In [34]:
import pandas as pd
import numpy as np
import json
import os
import re
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import string

a)remove all duplicates from sheet in excel using DATA>remove duplicates
b)using excel concatenate function joined "AgentCorrected_CatName_Primary" and "AgentCorrected_Integer_Primary" column with comma and new target column name is "primary".


In [35]:

df2_cicd=pd.read_csv('../data/TaxML-CICD - Prod_Data_0501.csv')

In [36]:
def preprocess_text(message):

    new_stopwords=['ml','oz','pk','grocery','lb']
    stpwrd = nltk.corpus.stopwords.words('english')
    stpwrd.extend(new_stopwords)
    # 1. Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
    #lowering and removing punctuation
    message = re.sub(r'[^\w\s]','', message.lower())
    #removing the numerical values and working only with text values
    message = re.sub('[^a-zA-Z]', " ", message )
    #removing the stopwords
    message = ' '.join([word for word in message.split() if word not in stpwrd and len(word)>1])
    #lemmatizing the text
    message =  " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(message) if w not in string.punctuation])
    #print("message is : ",message)
    return message

In [37]:
df = pd.read_csv('../data/jina_final.csv', encoding='utf8',engine='python',usecols=['Item','Description','establishment_type','primary','secondary'])
df1=df.sample(frac=1, random_state=42)
df1 = df1.fillna('')
df1['input_str'] = df1[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df1['cleanText']=df1['input_str'].map(lambda s:preprocess_text(s)) 
df1 = df1.reset_index(drop=True)
X=df1[['Item','Description','establishment_type','cleanText']]
Y_primary=df1['primary']
#Train test split with stratified sampling for evaluation
X_train, X_test, y_train_primary, y_test_primary = train_test_split(X,
                                                    Y_primary,
                                                    test_size = .20, 
                                                    random_state = 42
                                                    )

In [38]:
df.head()

,Item,Description,establishment_type,primary,secondary
0,Captain Morgan White Rum.1.75L Bottle,NaN,GROCERY,"CAT_LIQUOR,535","CAT_LIQUOR,535"
1,D’USSÉ® VSOP Cognac.375ml Bottle,NaN,GROCERY,"CAT_LIQUOR,535","CAT_LIQUOR,535"
2,Jim Beam Honey Bourbon Whiskey.750ml Bottle,NaN,GROCERY,"CAT_LIQUOR,535","CAT_LIQUOR,535"
3,Ardbeg Scotch Uigeadail. 750ml Bottle,NaN,GROCERY,"CAT_LIQUOR,535","CAT_LIQUOR,535"
4,Jameson Irish Whiskey.1.75L Bottle Size,NaN,GROCERY,"CAT_LIQUOR,535","CAT_LIQUOR,535"


In [39]:
Y_secondary =df1['secondary']
#Train test split with stratified sampling for evaluation
X_train, X_test, y_train_secondary, y_test_secondary = train_test_split(X,
                                                    Y_secondary,
                                                    test_size = .20, 
                                                    random_state = 42
                                                    )

In [40]:
# df2_cicd.head()

In [41]:
len(df2_cicd)

16635

In [42]:
df2 = df2_cicd
df2.drop_duplicates(inplace=True)

In [43]:
len(df2)

13919

In [44]:
df2.dropna(how='all',inplace=True)

In [45]:
len(df2)

13918

In [46]:
# df2.loc[df2['AgentCorrected_Integer_Primary'].isnull()]

In [47]:
df2['primary'] = df2['AgentCorrected_CatName_Primary'] + ',' + df2['AgentCorrected_Integer_Primary'].astype('int').astype('str')

In [48]:
df2.loc[5,'primary']

'CAT_PREPARED_FOOD,101'

In [49]:
# df2 = df2_cicd
df2['input_str'] = df2[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df2['cleanText']=df2['input_str'].map(lambda s:preprocess_text(s))
X_cicd=df2[['Item','Description','establishment_type','cleanText']]

Y_primary_cicd=df2['primary']
X_train_cicd, X_test_cicd, y_train_primary_cicd, y_test_primary_cicd = train_test_split(X_cicd,
                                                    Y_primary_cicd,
                                                    test_size = .20, 
                                                    random_state = 42
                                                    )
X_train_final_1 = X_train.append(X_train_cicd)
X_test_final_1 = X_test.append(X_test_cicd)
X_train_final=X_train['cleanText'].append(X_train_cicd['cleanText'])
y_train_final_primary=y_train_primary.append(y_train_primary_cicd)
X_test_final=X_test['cleanText'].append(X_test_cicd['cleanText'])
y_test_final_primary=y_test_primary.append(y_test_primary_cicd)

In [133]:
df2

,CICD Run Date,Date (BOT Sent Details to COE Team),UniqueUUID,store_uuid,item_uuid,Item,Description,establishment_type,CAT Name_Primary_ConfScore,CAT Name_Primary,...,AgentCorrected_CatName_Secondary2.1,Integer_Secondary2_ValidationScore,Integer_Avg_ValidationScore,Integer_Avg_ValidationScore.1,primary,input_str,cleanText,new_secondary_catname,new_secondary_integer,secondary
0,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:4dbee795-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,4dbee795-530a-4802-8dde-a5b28d51d4d9,Philly Cheese Steak,NaN,GROCERY,0.85,CAT_PREPARED_FOOD,...,NaN,100.0,NaN,NaN,"CAT_PREPARED_FOOD,101",Philly Cheese Steak GROCERY,philly cheese steak,CAT_PREPARED_FOOD,101,"CAT_PREPARED_FOOD,101"
1,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:f090eff8-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,f090eff8-7723-4b19-b464-b8479497f52f,Mango Smoothie,NaN,GROCERY,0.75,CAT_PREPARED_DRINK,...,NaN,100.0,NaN,NaN,"CAT_PREPARED_DRINK,114",Mango Smoothie GROCERY,mango smoothie,CAT_PREPARED_DRINK,114,"CAT_PREPARED_DRINK,114"
2,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:f9423393-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,f9423393-f715-42af-a6c0-1e5acec8f120,Banana Strawberry Smoothie,NaN,GROCERY,0.85,CAT_PREPARED_DRINK,...,NaN,100.0,NaN,NaN,"CAT_PREPARED_DRINK,114",Banana Strawberry Smoothie GROCERY,banana strawberry smoothie,CAT_PREPARED_DRINK,114,"CAT_PREPARED_DRINK,114"
3,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:0a1a42e9-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,0a1a42e9-c63f-4407-8d2e-f370f7f4d26d,"Fruits Salad Mango, Banana, Strwberry, and Ora...",NaN,GROCERY,0.30,CAT_PREPARED_FOOD_PREPARED_SALADS,...,NaN,100.0,NaN,NaN,"CAT_PREPARED_DRINK,114","Fruits Salad Mango, Banana, Strwberry, and Ora...",fruit salad mango banana strwberry orange smoo...,CAT_PREPARED_DRINK,114,"CAT_PREPARED_DRINK,114"
4,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:3977c4db-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,3977c4db-0e3d-4c39-bc1e-536cfffe0603,Tea,"Chamomile green tea, lipton, raspberry, or gin...",GROCERY,0.50,CAT_PREPARED_DRINK,...,NaN,100.0,NaN,NaN,"CAT_PREPARED_DRINK,114","Tea Chamomile green tea, lipton, raspberry, or...",tea chamomile green tea lipton raspberry ginger,CAT_PREPARED_DRINK,114,"CAT_PREPARED_DRINK,114"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16626,04-01-2022 9:52 PM,03-01-2022 4:08 AM,3dd472e4-853c-576e-8cfc-427ca8e4aee3:23f8af40-...,3dd472e4-853c-576e-8cfc-427ca8e4aee3,23f8af40-a2e2-4cf3-983c-1e896680e024,Coca Cola (12 oz),NaN,CONVENIENCE,1.00,CAT_SOFT_DRINK,...,1.0,0.0,NaN,NaN,"CAT_SOFT_DRINK,112",Coca Cola (12 oz) CONVENIENCE,coca cola convenience,CONTAINER_BOTTLED,11,"CONTAINER_BOTTLED,11"
16627,04-01-2022 9:52 PM,03-01-2022 4:08 AM,3dd472e4-853c-576e-8cfc-427ca8e4aee3:21188247-...,3dd472e4-853c-576e-8cfc-427ca8e4aee3,21188247-fd37-406c-9cc6-21a6b1a924a2,Sour Patch (2 oz),NaN,CONVENIENCE,1.00,CAT_CANDY,...,NaN,100.0,NaN,NaN,"CAT_CANDY,108",Sour Patch (2 oz) CONVENIENCE,sour patch convenience,CAT_CANDY,108,"CAT_CANDY,108"
16628,04-01-2022 9:52 PM,03-01-2022 4:08 AM,3dd472e4-853c-576e-8cfc-427ca8e4aee3:bd635383-...,3dd472e4-853c-576e-8cfc-427ca8e4aee3,bd635383-c925-42e5-ad35-0aef40d856d0,Canada Dry Ginger Ale (12 oz),NaN,CONVENIENCE,1.00,CAT_SOFT_DRINK,...,1.0,0.0,NaN,NaN,"CAT_SOFT_DRINK,112",Canada Dry Ginger Ale (12 oz) CONVENIENCE,canada dry ginger ale convenience,CONTAINER_BOTTLED,11,"CONTAINER_BOTTLED,11"
16629,04-01-2022 9:52 PM,03-01-2022 4:08 AM,3dd472e4-853c-576e-8cfc-427ca8e4aee3:6bf4ebdd-...,3dd472e4-853c-576e-8cfc-427ca8e4aee3,6bf4ebdd-fa37-4e62-bf6e-742eb2299583,Fiji (500 ml),NaN,CONVENIENCE,0.80,CAT_WATER,...,NaN,100.0,NaN,NaN,"CAT_WATER,111",Fiji (500 ml) CONVENIENCE,fiji convenience,CAT_WATER,111,"CAT_WATER,111"


split and save  data for train and test

In [141]:
train_size = 0.8
train_end = int(len(df1)*train_size)
df_train = df1[:train_end]
df_test = df1[train_end:]
train_end_cicd = int(len(df2)*train_size)
df2_train = df2[:train_end_cicd]
df2_test = df2[train_end_cicd:]
df1_train = df_train[['Item','Description','establishment_type','primary','input_str','cleanText']]
df1_test = df_test[['Item','Description','establishment_type','primary','input_str','cleanText']]
df2_train = df2_train[['Item','Description','establishment_type','primary','input_str','cleanText']]
df2_test = df2_test[['Item','Description','establishment_type','primary','input_str','cleanText']]
X_train_save = df1_train.append(df2_train)
X_test_save = df1_test.append(df2_test)
X_train_save['label'] = 'train'
X_test_save['label'] = 'test'
X_data = X_train_save.append(X_test_save)
X_data.to_csv('df_traintestdata_primary_06-01-22.csv')

In [142]:
X_data

,Item,Description,establishment_type,primary,input_str,cleanText,label
0,"Aviation Gin, 750mL gin (42.0% ABV)",Seven flavoring ingredients are used in its pr...,GROCERY,"CAT_LIQUOR,535","Aviation Gin, 750mL gin (42.0% ABV) Seven flav...",aviation gin gin abv seven flavoring ingredien...,train
1,Smoked Turkey Breast Sandwich,,GROCERY,"CAT_PREPARED_FOOD,101",Smoked Turkey Breast Sandwich GROCERY,smoked turkey breast sandwich,train
2,Cotes Du Rhone Reserve Faille Perrin 750ml,\N,GROCERY,"CAT_WINE,534",Cotes Du Rhone Reserve Faille Perrin 750ml \N ...,cote du rhone reserve faille perrin,train
3,Schramsberg Blanc de Blancs,Blanc de Blancs (white from white) made from C...,GROCERY,"CAT_SPARKLING_WINE,716",Schramsberg Blanc de Blancs Blanc de Blancs (w...,schramsberg blanc de blanc blanc de blanc whit...,train
4,Cortas Lebanese Extra Virgin Olive Oil (1.875 lb),,GROCERY,"CAT_PREPACKAGED_FOOD_EDIBLE_OILS,727",Cortas Lebanese Extra Virgin Olive Oil (1.875 ...,cortas lebanese extra virgin olive oil,train
...,...,...,...,...,...,...,...
16626,Coca Cola (12 oz),NaN,CONVENIENCE,"CAT_SOFT_DRINK,112",Coca Cola (12 oz) CONVENIENCE,coca cola convenience,test
16627,Sour Patch (2 oz),NaN,CONVENIENCE,"CAT_CANDY,108",Sour Patch (2 oz) CONVENIENCE,sour patch convenience,test
16628,Canada Dry Ginger Ale (12 oz),NaN,CONVENIENCE,"CAT_SOFT_DRINK,112",Canada Dry Ginger Ale (12 oz) CONVENIENCE,canada dry ginger ale convenience,test
16629,Fiji (500 ml),NaN,CONVENIENCE,"CAT_WATER,111",Fiji (500 ml) CONVENIENCE,fiji convenience,test


In [51]:
X_train_final_1

,Item,Description,establishment_type,cleanText
74328,"Exotico Blanco, 750mL bottle (40% ABV)",,GROCERY,exotico blanco bottle abv
93071,Cortas Halva Original (16 oz),,GROCERY,cortas halva original
160229,"Meiomi Pinot Noir, 750mL wine (13.7% ABV)",,GROCERY,meiomi pinot noir wine abv
33442,Ito en Peach Veggie Shot,30.36 fluid ounces.,GROCERY,ito en peach veggie shot fluid ounce
129506,"Val Di Giulia Barbaresco, 750mL italian red wi...",,GROCERY,val di giulia barbaresco italian red wine abv
...,...,...,...,...
5856,"Cazadores Tequila Reposado 750mL, 1 tequila an...","91 pts beverage testing institute. Pale, silve...",LIQUOR,cazadores tequila reposado tequila mezcal abv ...
16067,"Major Peters' Cocktail Margarita, 1L bottle mi...",NaN,LIQUOR,major peter cocktail margarita bottle mixer ab...
6120,American Dream Roll,"Spicy tuna, salmon, eel, shrimp, avocado, and ...",DEFAULT,american dream roll spicy tuna salmon eel shri...
904,"Vita Salmon, Nova, Atlantic, Premium, Sliced, ...",NaN,GROCERY,vita salmon nova atlantic premium sliced smoked


In [52]:
result=X_test_final_1

rf = Pipeline([('vect', CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', lowercase=True, stop_words='english')),
       ('tfidf', TfidfTransformer(use_idf=True)),
       ('clf', RandomForestClassifier(oob_score=True,n_jobs=-1)),
      ])
rf.fit(X_train_final, y_train_final_primary)
y_pred= rf.predict(X_test_final)
result['original_cat_primary']=y_test_final_primary
result['prediction_cat_primary']=y_pred
result['prediction_cat_primary_confscore']=rf.predict_proba(X_test_final).max()
output={'accuracy':accuracy_score(y_pred,y_test_final_primary),'precision_score':precision_score(y_pred,y_test_final_primary,average='macro'),'recall_score':recall_score(y_pred,y_test_final_primary,average='macro')
,'f1_score':f1_score(y_pred,y_test_final_primary,average='macro')}
result['confusion_matrix_primary']=str(output)



/Users/ssingh355/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/ssingh355/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/ssingh355/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/ssingh355/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users

In [53]:
result

,Item,Description,establishment_type,cleanText,original_cat_primary,prediction_cat_primary,prediction_cat_primary_confscore,confusion_matrix_primary
12184,Dungeons and Dragons 5E Candle Keep Mysteries,Great books hide their secrets well. An anthol...,GROCERY,dungeon dragon candle keep mystery great book ...,"CAT_TPP_TOYS,822","CAT_TPP,531",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
142293,Tam Fruit Company-Fresh Cut Celery Sticks10 oz,,GROCERY,tam fruit companyfresh cut celery stick,"CAT_FRUIT_VEG_PLANTS,552","CAT_PREPACKAGED_FOOD_FRESH_FRUITS,752",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
44634,Burnetts Vodka 1.75L 40% ABV,\N,GROCERY,burnett vodka abv,"CAT_LIQUOR,535","CAT_LIQUOR,535",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
136489,"Llano ""Cab Sav"", 750mL wine (13.2% ABV)",,GROCERY,llano cab sav wine abv,"CAT_WINE,534","CAT_WINE,534",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
44540,Emergen-C-Super Orange30 ct,,GROCERY,emergencsuper orange ct,"CAT_SUPPLEMENTS,542","CAT_FRUIT_VEG_PLANTS,552",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
...,...,...,...,...,...,...,...,...
12910,"Sailor Jerry Spiced Rum, 750mL (46% ABV)",Smoothly spicy with notes of vanilla and cinna...,LIQUOR,sailor jerry spiced rum abv smoothly spicy not...,"CAT_LIQUOR,535","CAT_LIQUOR,535",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
1971,Leader Mineral Oil USP Lubricant Laxative (16 ...,NaN,PHARMACY,leader mineral oil usp lubricant laxative fl p...,"CAT_OTC_MEDICATION_LAXATIVES,769","CAT_OTC_MEDICATION_LAXATIVES,769",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
1811,Aquacolor Set 18 Colors,NaN,SPECIALITY_STORE,aquacolor set color speciality store,"CAT_TPP_COSMETICS,783","CAT_TPP_COSMETICS,783",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
13517,Bridgford Pepperoni (5 oz),NaN,GROCERY,bridgford pepperoni,"CAT_PREPACKAGED_FOOD,106","CAT_PREPACKAGED_FOOD,106",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."


In [54]:
misclassifications_primary= result.loc[result['original_cat_primary']!=result['prediction_cat_primary']]

In [55]:
misclassifications_primary

,Item,Description,establishment_type,cleanText,original_cat_primary,prediction_cat_primary,prediction_cat_primary_confscore,confusion_matrix_primary
12184,Dungeons and Dragons 5E Candle Keep Mysteries,Great books hide their secrets well. An anthol...,GROCERY,dungeon dragon candle keep mystery great book ...,"CAT_TPP_TOYS,822","CAT_TPP,531",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
142293,Tam Fruit Company-Fresh Cut Celery Sticks10 oz,,GROCERY,tam fruit companyfresh cut celery stick,"CAT_FRUIT_VEG_PLANTS,552","CAT_PREPACKAGED_FOOD_FRESH_FRUITS,752",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
44540,Emergen-C-Super Orange30 ct,,GROCERY,emergencsuper orange ct,"CAT_SUPPLEMENTS,542","CAT_FRUIT_VEG_PLANTS,552",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
107027,Kyoshin Fruit Daifuku,"Mango, peach and pineapple. 10.58 oz. 300 gram...",GROCERY,kyoshin fruit daifuku mango peach pineapple gr...,"CAT_PREPACKAGED_FOOD,106","CAT_CANDY,108",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
57415,Evo Fabuloso Colour Intensifying Platinum Cond...,Extend the life of your color-treated ‘do in b...,SPECIALITY_STORE,evo fabuloso colour intensifying platinum cond...,"CAT_TPP_SHAMPOOS,816","CAT_TPP,531",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
...,...,...,...,...,...,...,...,...
7828,Hormel Chili with bean,NaN,CONVENIENCE,hormel chili bean convenience,"CAT_PREPACKAGED_FOOD_SHELF_STABLE_SOUP,744","CAT_PREPACKAGED_FOOD,106",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
8353,Met/RX (3.52 oz),NaN,CONVENIENCE,metrx convenience,"CAT_PREPACKAGED_FOOD_SNACK_SNACK_BARS,748","CAT_BREATH_MINTS,559",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
11681,Radicchio (each),NaN,GROCERY,radicchio,"CAT_PREPACKAGED_FOOD,106","CAT_SOFT_DRINK,112",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."
4414,Creamy Hazelnut Cinnamon (1 lb),NaN,GROCERY,creamy hazelnut cinnamon,"CAT_PREPACKAGED_FOOD,106","CAT_PREPACKAGED_FOOD_NUT_SPREADS,735",1.0,"{'accuracy': 0.8205476844423898, 'precision_sc..."


In [56]:
output

{'accuracy': 0.8205476844423898,
 'precision_score': 0.5986736734285344,
 'recall_score': 0.686097316633149,
 'f1_score': 0.6257994957846045}

In [57]:

# misclassifications_primary= result.loc[result['original_cat_primary']!=result['prediction_cat_primary']]

"{'accuracy': 0.8146149522203485, 'precision_score': 0.6027551606729535, 'recall_score': 0.7097466761674291, 'f1_score': 0.6327463859326979}"

saving model to local

In [58]:
import pickle
# save the model to disk
filename_primary= 'finalized_model_rf_primary-06-01-22_allprod.sav'
pickle.dump(rf, open(filename_primary, 'wb'))


Processing Secondary column data

In [90]:
def process_secondary_catname(category):
    
    if category != None:
    
        if category in ['TEMP_HEATED','TEMP_COLD','TEMP_UNHEATED']:
            category= None
    
    return category
    
def process_secondary_integer(secondary_integer):
    
    if secondary_integer != None:
        secondary_integer = str(int(float(secondary_integer)))
        if (len(secondary_integer)==3 and secondary_integer not in ['105','107']) or secondary_integer == '1' or secondary_integer == '2':      
            secondary_integer = None

    return secondary_integer
        

In [91]:
df2_selected_secondary = df2

In [92]:
# df2['AgentCorrected_Integer_Secondary'].value_counts()

In [93]:
# df2_selected_secondary['AgentCorrected_Integer_Secondary'] = df2_selected_secondary['AgentCorrected_Integer_Secondary'].astype(int)

In [94]:
df2_selected_secondary['AgentCorrected_Integer_Secondary'] = df2_selected_secondary['AgentCorrected_Integer_Secondary'].apply(lambda x : process_secondary_integer(x))

In [95]:
df2_selected_secondary['AgentCorrected_Integer_Secondary'].isnull().sum()

12641

In [96]:
df2_selected_secondary['AgentCorrected_Integer_Secondary'].value_counts()

11     835
107    177
105     93
51      86
61      82
53       4
Name: AgentCorrected_Integer_Secondary, dtype: int64

In [97]:
df2_selected_secondary['AgentCorrected_CatName_Secondary'] = df2_selected_secondary['AgentCorrected_CatName_Secondary'].apply(lambda x : process_secondary_catname(x))

In [98]:
df2_selected_secondary['AgentCorrected_CatName_Secondary'].isnull().sum()

12593

In [99]:
df2_selected_secondary.loc[df2_selected_secondary['AgentCorrected_Integer_Secondary'].isnull(),'AgentCorrected_CatName_Secondary' ] = None

In [100]:
df2_selected_secondary['AgentCorrected_CatName_Secondary'].isnull().sum()

12641

In [101]:
df2_selected_secondary['new_secondary_catname'] = df2_selected_secondary['AgentCorrected_CatName_Secondary'].fillna(df2_selected_secondary['AgentCorrected_CatName_Primary'])

In [103]:
df2_selected_secondary[10:20]

,CICD Run Date,Date (BOT Sent Details to COE Team),UniqueUUID,store_uuid,item_uuid,Item,Description,establishment_type,CAT Name_Primary_ConfScore,CAT Name_Primary,...,Integer_Secondary_ValidationScore,Integer_Secondary2,AgentCorrected_CatName_Secondary2.1,Integer_Secondary2_ValidationScore,Integer_Avg_ValidationScore,Integer_Avg_ValidationScore.1,primary,input_str,cleanText,new_secondary_catname
11,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:6faa1442-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,6faa1442-cb85-4152-9feb-37857a789e0e,Buffalo Wings and Fries,NaN,GROCERY,1.00,CAT_PREPARED_FOOD,...,100.0,NaN,NaN,100.0,NaN,NaN,"CAT_PREPARED_FOOD,101",Buffalo Wings and Fries GROCERY,buffalo wing fry,CAT_PREPARED_FOOD
12,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:a986f1e1-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,a986f1e1-e0cf-48ab-8ba4-4e02e2c3bf33,Red Bull,NaN,GROCERY,0.98,CAT_ENERGY_DRINK,...,100.0,NaN,NaN,100.0,NaN,NaN,"CAT_ENERGY_DRINK,113",Red Bull GROCERY,red bull,CONTAINER_BOTTLED
13,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:87bb0259-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,87bb0259-3607-46cb-9342-038468987826,Coconut Lemonade,NaN,GROCERY,0.35,CAT_PREPARED_DRINK,...,100.0,NaN,NaN,100.0,NaN,NaN,"CAT_JUICE_NON_CARBONATED,537",Coconut Lemonade GROCERY,coconut lemonade,CAT_JUICE_NON_CARBONATED
14,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:ff959757-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,ff959757-9339-4452-806c-6f6e5abe8316,Ham Egg and Cheese Sandwich,NaN,GROCERY,0.93,CAT_PREPARED_FOOD,...,100.0,NaN,NaN,100.0,NaN,NaN,"CAT_PREPARED_FOOD,101",Ham Egg and Cheese Sandwich GROCERY,ham egg cheese sandwich,CAT_PREPARED_FOOD
15,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:e77fdd72-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,e77fdd72-c44b-4f14-b28e-3b31e0a631fc,Morir Sonando,NaN,GROCERY,0.49,CAT_SOFT_DRINK,...,0.0,NaN,NaN,100.0,NaN,NaN,"CAT_PREPARED_DRINK,114",Morir Sonando GROCERY,morir sonando,CAT_PREPARED_DRINK
16,28-12-2021 12:52 AM,NaN,61ff9153-b582-5003-8680-466f512bb8e1:0a8b1f52-...,61ff9153-b582-5003-8680-466f512bb8e1,0a8b1f52-7aa2-47d6-99d4-1721148c331a,Kalamata Jumbo Olives (1 lb),NaN,GROCERY,0.60,CAT_PREPACKAGED_FOOD_PICKLES,...,100.0,NaN,NaN,100.0,NaN,NaN,"CAT_PREPACKAGED_FOOD_PICKLES,736",Kalamata Jumbo Olives (1 lb) GROCERY,kalamata jumbo olive,CAT_PREPACKAGED_FOOD_PICKLES
17,28-12-2021 12:52 AM,NaN,8fed2af5-d154-444a-aa96-0992a845c008:8314bf7d-...,8fed2af5-d154-444a-aa96-0992a845c008,8314bf7d-88f6-4ce8-91c5-951fc66efe28,Organic Thyme (1 bunch),NaN,GROCERY,0.60,CAT_FRUIT_VEG_PLANTS,...,100.0,NaN,NaN,100.0,NaN,NaN,"CAT_FRUIT_VEG_PLANTS,552",Organic Thyme (1 bunch) GROCERY,organic thyme bunch,CAT_FRUIT_VEG_PLANTS
18,28-12-2021 12:52 AM,NaN,678e0859-2548-511a-9b1e-eae65c73a6b0:68934166-...,678e0859-2548-511a-9b1e-eae65c73a6b0,68934166-131a-4e23-ba5a-1ad8cee6831b,Aquaphor Advanced Protection (1.75 oz),NaN,GROCERY,0.50,CAT_MEDICATED_ITEMS,...,100.0,NaN,NaN,100.0,NaN,NaN,"CAT_MEDICATED_ITEMS,525",Aquaphor Advanced Protection (1.75 oz) GROCERY,aquaphor advanced protection,CAT_MEDICATED_ITEMS
19,28-12-2021 12:52 AM,NaN,678e0859-2548-511a-9b1e-eae65c73a6b0:1eeb3417-...,678e0859-2548-511a-9b1e-eae65c73a6b0,1eeb3417-4b5b-445e-ab9e-51c603300ab6,Unisom Sleepgels (8 ct),NaN,GROCERY,0.85,CAT_OTC_MEDICATION,...,100.0,NaN,NaN,100.0,NaN,NaN,"CAT_OTC_MEDICATION,521",Unisom Sleepgels (8 ct) GROCERY,unisom sleepgels ct,CAT_OTC_MEDICATION
20,28-12-2021 12:52 AM,NaN,678e0859-2548-511a-9b1e-eae65c73a6b0:7f42d994-...,678e0859-2548-511a-9b1e-eae65c73a6b0,7f42d994-b839-4b68-b79e-dacb1eb711d4,Advil Coated Caplets (50 ct),NaN,GROCERY,1.00,CAT_OTC_MEDICATION_IBUPROFEN,...,100.0,NaN,NaN,100.0,NaN,NaN,"CAT_OTC_MEDICATION_IBUPROFEN,768",Advil Coated Caplets (50 ct) GROCERY,advil coated caplets ct,CAT_OTC_MEDICATION_IBUPROFEN


In [104]:
df2_selected_secondary['new_secondary_integer'] = df2_selected_secondary['AgentCorrected_Integer_Secondary'].fillna(df2_selected_secondary['AgentCorrected_Integer_Primary'])

In [105]:
# df2_selected_secondary = df2.dropna(subset=['AgentCorrected_Integer_Secondary'])

In [106]:
df2_selected_secondary['new_secondary_integer'] = df2_selected_secondary['new_secondary_integer'].astype(int)

In [120]:
df2_selected_secondary['new_secondary_integer'] = df2_selected_secondary['new_secondary_integer'].astype(str)

In [107]:
# df2_selected_secondary['Integer_Secondary_processed'] = df2_selected_secondary['AgentCorrected_Integer_Secondary'].apply(lambda x : process_secondary_integer(x))

In [108]:
len(df2_selected_secondary)

13918

In [110]:
# df2_selected_secondary['Integer_Secondary_processed'].isnull().sum()

In [111]:
# df2_selected_secondary.dropna(subset=['Integer_Secondary_processed'], inplace = True)

In [113]:
# len(df2_selected_secondary)

In [114]:
# df2_selected_secondary['Secondary_catname_processed'] = df2_selected_secondary['AgentCorrected_CatName_Secondary'].apply(lambda x : process_secondary_catname(x))

In [115]:
# df2_selected_secondary['Secondary_catname_processed'].isnull().sum()

In [116]:
# df2_selected_secondary.dropna(subset=['Secondary_catname_processed'], inplace = True)

In [117]:
# len(df2_selected_secondary)

In [118]:
# df2_selected_secondary['Integer_Secondary_processed'].value_counts()

In [121]:
df2_selected_secondary['secondary'] = df2_selected_secondary['new_secondary_catname'] + ','+ df2_selected_secondary['new_secondary_integer']

In [124]:
df2_selected_secondary[10:15]

,CICD Run Date,Date (BOT Sent Details to COE Team),UniqueUUID,store_uuid,item_uuid,Item,Description,establishment_type,CAT Name_Primary_ConfScore,CAT Name_Primary,...,AgentCorrected_CatName_Secondary2.1,Integer_Secondary2_ValidationScore,Integer_Avg_ValidationScore,Integer_Avg_ValidationScore.1,primary,input_str,cleanText,new_secondary_catname,new_secondary_integer,secondary
11,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:6faa1442-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,6faa1442-cb85-4152-9feb-37857a789e0e,Buffalo Wings and Fries,NaN,GROCERY,1.00,CAT_PREPARED_FOOD,...,NaN,100.0,NaN,NaN,"CAT_PREPARED_FOOD,101",Buffalo Wings and Fries GROCERY,buffalo wing fry,CAT_PREPARED_FOOD,101,"CAT_PREPARED_FOOD,101"
12,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:a986f1e1-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,a986f1e1-e0cf-48ab-8ba4-4e02e2c3bf33,Red Bull,NaN,GROCERY,0.98,CAT_ENERGY_DRINK,...,NaN,100.0,NaN,NaN,"CAT_ENERGY_DRINK,113",Red Bull GROCERY,red bull,CONTAINER_BOTTLED,11,"CONTAINER_BOTTLED,11"
13,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:87bb0259-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,87bb0259-3607-46cb-9342-038468987826,Coconut Lemonade,NaN,GROCERY,0.35,CAT_PREPARED_DRINK,...,NaN,100.0,NaN,NaN,"CAT_JUICE_NON_CARBONATED,537",Coconut Lemonade GROCERY,coconut lemonade,CAT_JUICE_NON_CARBONATED,537,"CAT_JUICE_NON_CARBONATED,537"
14,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:ff959757-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,ff959757-9339-4452-806c-6f6e5abe8316,Ham Egg and Cheese Sandwich,NaN,GROCERY,0.93,CAT_PREPARED_FOOD,...,NaN,100.0,NaN,NaN,"CAT_PREPARED_FOOD,101",Ham Egg and Cheese Sandwich GROCERY,ham egg cheese sandwich,CAT_PREPARED_FOOD,101,"CAT_PREPARED_FOOD,101"
15,28-12-2021 12:52 AM,NaN,63d0485d-3a81-5b24-bf0a-1137d4bad610:e77fdd72-...,63d0485d-3a81-5b24-bf0a-1137d4bad610,e77fdd72-c44b-4f14-b28e-3b31e0a631fc,Morir Sonando,NaN,GROCERY,0.49,CAT_SOFT_DRINK,...,NaN,100.0,NaN,NaN,"CAT_PREPARED_DRINK,114",Morir Sonando GROCERY,morir sonando,CAT_PREPARED_DRINK,114,"CAT_PREPARED_DRINK,114"


In [125]:
df2_selected_secondary['input_str'] = df2_selected_secondary[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df2_selected_secondary['cleanText']=df2_selected_secondary['input_str'].map(lambda s:preprocess_text(s))
X_cicd2=df2_selected_secondary[['Item','Description','establishment_type','cleanText']]

Y_secondary_cicd=df2_selected_secondary['secondary']
X_train_cicd2, X_test_cicd2, y_train_secondary_cicd2, y_test_secondary_cicd2 = train_test_split(X_cicd2,
                                                    Y_secondary_cicd,
                                                    test_size = .20, 
                                                    random_state = 42
                                                    )

X_train_final_2 = X_train.append(X_train_cicd2)
X_test_final_2 = X_test.append(X_test_cicd2)
X_train_final_secondary =X_train['cleanText'].append(X_train_cicd2['cleanText'])
y_train_final_secondary =y_train_secondary.append(y_train_secondary_cicd2)
X_test_final_secondary =X_test['cleanText'].append(X_test_cicd2['cleanText'])
y_test_final_secondary = y_test_secondary.append(y_test_secondary_cicd2)



In [126]:
result=X_test_final_2

rf = Pipeline([('vect', CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', lowercase=True, stop_words='english')),
       ('tfidf', TfidfTransformer(use_idf=True)),
       ('clf', RandomForestClassifier(oob_score=True,n_jobs=-1)),
      ])
rf.fit(X_train_final_secondary, y_train_final_secondary)
y_pred= rf.predict(X_test_final_secondary)
result['original_cat_secondary']=y_test_final_secondary
result['prediction_cat_secondary']=y_pred
result['prediction_cat_primary_confscore']=rf.predict_proba(X_test_final_secondary).max()
output={'accuracy':accuracy_score(y_pred,y_test_final_secondary),'precision_score':precision_score(y_pred,y_test_final_secondary,average='macro'),'recall_score':recall_score(y_pred,y_test_final_secondary,average='macro')
,'f1_score':f1_score(y_pred,y_test_final_secondary,average='macro')}
result['confusion_matrix_secondary']=str(output)


/Users/ssingh355/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/ssingh355/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/ssingh355/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/ssingh355/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users

In [127]:
output

{'accuracy': 0.816087890571887,
 'precision_score': 0.6073885249266578,
 'recall_score': 0.7074847529636719,
 'f1_score': 0.635212341849785}

In [128]:
# save the model to disk
filename_secondary = 'finalized_model_rf_secondary-06-01-22_allprod.sav'
pickle.dump(rf, open(filename_secondary, 'wb'))


In [132]:
result.loc[result['Item'] == 'Sprite Mixer']

,Item,Description,establishment_type,cleanText,original_cat_secondary,prediction_cat_secondary,prediction_cat_primary_confscore,confusion_matrix_secondary
15546,Sprite Mixer,NaN,CONVENIENCE,sprite mixer convenience,"CONTAINER_BOTTLED,11","CONTAINER_BOTTLED,11",1.0,"{'accuracy': 0.816087890571887, 'precision_sco..."
16122,Sprite Mixer,NaN,GROCERY,sprite mixer,"CONTAINER_BOTTLED,11","CONTAINER_BOTTLED,11",1.0,"{'accuracy': 0.816087890571887, 'precision_sco..."
2658,Sprite Mixer,NaN,GROCERY,sprite mixer,"CONTAINER_BOTTLED,11","CONTAINER_BOTTLED,11",1.0,"{'accuracy': 0.816087890571887, 'precision_sco..."
12503,Sprite Mixer,NaN,GROCERY,sprite mixer,"CONTAINER_BOTTLED,11","CONTAINER_BOTTLED,11",1.0,"{'accuracy': 0.816087890571887, 'precision_sco..."


In [145]:
train_size = 0.8
train_end = int(len(df1)*train_size)
df_train = df1[:train_end]
df_test = df1[train_end:]
train_end_cicd = int(len(df2_selected_secondary)*train_size)
df1_train = df_train[['Item','Description','establishment_type','secondary','input_str','cleanText']]
df1_test = df_test[['Item','Description','establishment_type','secondary','input_str','cleanText']]
df2_train = df2_selected_secondary[:train_end_cicd]
df2_test = df2_selected_secondary[train_end_cicd:]
df2_train = df2_train[['Item','Description','establishment_type','secondary','input_str','cleanText']]
df2_test = df2_test[['Item','Description','establishment_type','secondary','input_str','cleanText']]
X_train_save = df1_train.append(df2_train)
X_test_save = df1_test.append(df2_test)
X_train_save['label'] = 'train'
X_test_save['label'] = 'test'
X_data = X_train_save.append(X_test_save)
X_data.to_csv('df_traintestdata_secondary_06-01-22.csv')

In [146]:
X_data

,Item,Description,establishment_type,secondary,input_str,cleanText,label
0,"Aviation Gin, 750mL gin (42.0% ABV)",Seven flavoring ingredients are used in its pr...,GROCERY,"CAT_LIQUOR,535","Aviation Gin, 750mL gin (42.0% ABV) Seven flav...",aviation gin gin abv seven flavoring ingredien...,train
1,Smoked Turkey Breast Sandwich,,GROCERY,"CAT_PREPARED_FOOD,101",Smoked Turkey Breast Sandwich GROCERY,smoked turkey breast sandwich,train
2,Cotes Du Rhone Reserve Faille Perrin 750ml,\N,GROCERY,"CAT_WINE,534",Cotes Du Rhone Reserve Faille Perrin 750ml \N ...,cote du rhone reserve faille perrin,train
3,Schramsberg Blanc de Blancs,Blanc de Blancs (white from white) made from C...,GROCERY,"CAT_SPARKLING_WINE,716",Schramsberg Blanc de Blancs Blanc de Blancs (w...,schramsberg blanc de blanc blanc de blanc whit...,train
4,Cortas Lebanese Extra Virgin Olive Oil (1.875 lb),,GROCERY,"CAT_PREPACKAGED_FOOD_EDIBLE_OILS,727",Cortas Lebanese Extra Virgin Olive Oil (1.875 ...,cortas lebanese extra virgin olive oil,train
...,...,...,...,...,...,...,...
16626,Coca Cola (12 oz),NaN,CONVENIENCE,"CONTAINER_BOTTLED,11",Coca Cola (12 oz) CONVENIENCE,coca cola convenience,test
16627,Sour Patch (2 oz),NaN,CONVENIENCE,"CAT_CANDY,108",Sour Patch (2 oz) CONVENIENCE,sour patch convenience,test
16628,Canada Dry Ginger Ale (12 oz),NaN,CONVENIENCE,"CONTAINER_BOTTLED,11",Canada Dry Ginger Ale (12 oz) CONVENIENCE,canada dry ginger ale convenience,test
16629,Fiji (500 ml),NaN,CONVENIENCE,"CAT_WATER,111",Fiji (500 ml) CONVENIENCE,fiji convenience,test


In [ ]:
# Sprite Mixer